# Creating candlestick charts for financial analysis

In this project, we are going to create candlestick plots which show the minute-by-minute activity of a particular stock. We are going to

- Load the relevant stock data
- Group the data in 15 minute increments
- For each increment, compute the open and close price
- For each increment, compute the min and max price over that 15 minute period
- Plot the bar plots, i.e, candlesticks, over time, overlayed with the min/max

To do this, we will need to import a few libraries for processing the data:

```python
import yfinance as yf
import pandas as pd
import numpy as np
```

To get you started, to download the historical data run

```python
stock_data = yf.download(tickers=STOCK_SYMBOL, period="1d", interval="1m")
```

where the ticker symbol can be anything, e.g., SPY, NVDA, etc. This will return the stock price at 1 minute intervals for the last day. The output will be a ```pandas.DataFrame``` with columns for the open, close, etc. prices, indexed by the datetime. Use the ```pandas.Grouper``` function to group the dataset in 15 minute increments and use the ```agg``` function to perform operations on those groups, e.g., to get the lowest or first price within that 15 minute interval, etc. 

In [71]:
%%html
<a href="candlestick.html">Candlestick Plot Example</a>

In [70]:
alt.renderers.enable('html')

RendererRegistry.enable('html')

In [3]:
# Import the yfinance. If you get module not found error the run !pip install yfinance from your Jupyter notebook
import yfinance as yf
import pandas as pd
import numpy as np

In [25]:
# get daily, yearly or hourly finance data
stock_symbol = "SPY"
stock_data = yf.download(tickers=stock_symbol, period="2d", interval="1m")

[*********************100%***********************]  1 of 1 completed


In [26]:
stock_data.head()

,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2024-05-29 09:30:00-04:00,525.679993,525.989990,525.599976,525.979980,525.979980,1502384
2024-05-29 09:31:00-04:00,525.979980,526.020020,525.789978,525.789978,525.789978,189565
2024-05-29 09:32:00-04:00,525.789978,525.849976,525.630005,525.780029,525.780029,155171
2024-05-29 09:33:00-04:00,525.789978,525.815002,525.544006,525.549988,525.549988,170719
2024-05-29 09:34:00-04:00,525.559998,525.669983,525.460022,525.614990,525.614990,195815


In [47]:
# filter yesterday only
import datetime
stock_data = stock_data[stock_data.index.date == datetime.date(2024, 5, 29)]

In [48]:
# data.groupby(pd.Grouper(freq='15Min')).size()
df = stock_data.reset_index().groupby(
    pd.Grouper(freq='15Min', key="Datetime")
).agg(
    open=("Open", "first"), 
    close=("Close", "last"),
    low=("Low", "min"),
    high=("High", "max"),
)

In [66]:
df["return"] = df["close"] - df["open"]
df.reset_index().head()

,Datetime,open,close,low,high,return
0,2024-05-29 09:30:00-04:00,525.679993,525.515015,525.424988,526.020020,-0.164978
1,2024-05-29 09:45:00-04:00,525.510010,526.200012,525.369995,526.419983,0.690002
2,2024-05-29 10:00:00-04:00,526.205017,526.400024,526.010010,526.543701,0.195007
3,2024-05-29 10:15:00-04:00,526.424988,526.729980,525.859985,526.969971,0.304993
4,2024-05-29 10:30:00-04:00,526.755981,526.929993,526.755981,527.304993,0.174011


In [63]:
# let's plot
import altair as alt
from vega_datasets import data

# source = data.ohlc()
# display(source.head())
source = df.reset_index()

open_close_color = alt.condition(
    "datum.open <= datum.close",
    alt.value("#06982d"),
    alt.value("#ae1325")
)

base = alt.Chart(source).encode(
    alt.X('Datetime:T')
        .axis(format='%H:%M', labelAngle=-80)
        .title('Datetime'),
    color=open_close_color
)

rule = base.mark_rule(size=1).encode(
    alt.Y('low:Q')
        .axis(titleAngle=0, titlePadding=20)
        .title(f'{stock_symbol}')
        .scale(zero=False),
    alt.Y2('high:Q'),
    tooltip=[
            alt.Tooltip("low:Q", format=",.3f"),
            alt.Tooltip("high:Q", format=",.3f")
    ]
)

bar = base.mark_bar(size=10).encode(
    alt.Y('open:Q'),
    alt.Y2('close:Q'),
    tooltip=[
        alt.Tooltip("close:Q", format=",.3f"),
        alt.Tooltip("open:Q", format=",.3f")
    ]
)

candlestick_plot = (rule+bar).properties(
    width=800,
    height=300
).configure_axis(
    labelFontSize=12,
    titleFontSize=12
)
candlestick_plot.interactive()

alt.LayerChart(...)

In [62]:
candlestick_plot.save("candlestick.html")

In [69]:
candlestick_plot.to_url()

'https://vega.github.io/editor/#/url/vega-lite/N4Igxg9gdgZglgcxALlANzgUwO4tJKAFzigFcJSBnAdTgBNCALFAZgAY2AacaYsiygAlMiRoVYcAvtwCGADziU8IADYyARphUAxXgGU4AL0woAjACZuxQisy6iB42fOTpqmQE9MAJxQBtUABbGW8Aa2VCDwAHE2QQb1JbEG5KI1jTN0woSDoSJFQeFQhfAoJc4mgIzEpxOLoZQlJAgDoIGKgAAgAeAF4O+saWsCLKE240GRVSWJAAYjYANgBOAA5zOhA3CamZ2ZlMUxZzAFZNqwgIFWIo-1B4LQ24otxuGGLg2uTmlhhkkEiYigQABHUgyIhwQgNOBoEzSO5YFSPECMUR-N7eD5AzjfX5WaIzUHg6zQ2GbAC63DkynkimUGKxcQApIJkEyALJ-NSaFQAQSgCCSyAAtCs2G57kigQARBqYYiBMb-SFCkCywjyuCKv4AmYawJRYqTM4gDw0hRKArWWz8wWxLjKmyYAAKMjouQFKHM4teiORzz+lDAk1ioGM3ggKBgk1GbmtMz0zoAmjqCUCiRCocQyW4POZ6X6gaiEMxXPCQMEwhE03F1CFA2kzD6QFkcnllJAiiV8NBynBKlbqp8Bk1Wu1un0R0MRkrttMgfNlmsNltJvO4nsDkdTnGLlc4DdkAEQJLkcMIKN0e8GtjcanAXEMyTs3DOAiHkC2lkr5ib3EcT896EmCmaknClIgNSBS0padzXp8LJspy3DclotpCqKzanjKcoKkq8Y4RqeFAUC+qGt4xq5gWH5xF+UAkY+IHPjCr6mvmBTYXE56XmWEEDDIyhQDI2p1A0MjCuYCwLCsdAAOwyCwmDmKYKzHMcpgLLJ0bHCwKxgCsCwACxqespiySajAiCWtTsA62D0EwKBig6AAkQaWcERaEIQUSUMgAD0-mwggMjNAgkKMKQ6jNP2-nuZgwRBZgIXCiokKYEFxzNGZ